# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
#pip install --upgrade langchain
!pip3 install -U pydantic==1.10.8

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [3]:
file = 'OutdoorClothingCatalog_3.csv'
loader = CSVLoader(file_path=file)

In [4]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
#pip install docarray

In [5]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

/Users/felixjin/.pyenv/versions/3.12.2/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


KeyboardInterrupt: 

In [ ]:
query ="Please list all your pants with sun protection \
in a table in markdown and summarize each one."

In [ ]:
response = index.query(query)

In [ ]:
display(Markdown(response))

In [11]:
loader = CSVLoader(file_path=file)

In [12]:
docs = loader.load()

In [13]:
docs[3]

Document(page_content=": 319\nname: Men's Tropical Breeze Pants\ndescription: These Men's cargo hiking pants are adventure-ready in a cool, moisture-wicking fabric that has 50+ UPF sunblock woven right in. Size & Fit - Classic Fit: Sits at the natural waist and trim, with a traditional straight leg. Fabric & Care - 100% nylon wicks moisture and resists wrinkles. Built-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. Additional Features - Button front with zip fly. Adjustable nylon belt. Two front slash pockets, two cargo pockets, two rear pockets. Zippered security pocket. Elastic inserts in waist. Imported.Sun Protection That Won't Wear Off - Our high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays.", metadata={'source': 'OutdoorClothingCatalog_3.csv', 'row': 3})

In [6]:
from langchain.embeddings import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002-felix",
    openai_api_version="2023-12-01-preview",
)
embeddings

/Users/felixjin/.pyenv/versions/3.12.2/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1300a9010>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x1300ad1f0>, model='text-embedding-ada-002', deployment='text-embedding-ada-002-felix', openai_api_version='2023-12-01-preview', openai_api_base=None, openai_api_type='azure', openai_proxy='', embedding_ctx_length=8191, openai_api_key='f49b792a6e60455f9237e607c9902d5b', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=16, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, azure_endpoint='https://openai-felix.openai.azure.com', azure_ad_token=None, azure_ad_token_provider=None, validate_base_url=True)

In [7]:
embed = embeddings.embed_query("Hi my name is Felix")


In [8]:
print(len(embed))

1536


In [9]:
print(embed[:5])

[-0.007811389272122033, 0.0031413307373589576, -0.011027099387213747, -0.0376263366544918, -0.02716894962874382]


In [14]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [15]:
query = "Please suggest a pant with sunblock"

In [16]:
docs = db.similarity_search(query)

In [17]:
len(docs)

4

In [18]:
docs[0]

Document(page_content=": 319\nname: Men's Tropical Breeze Pants\ndescription: These Men's cargo hiking pants are adventure-ready in a cool, moisture-wicking fabric that has 50+ UPF sunblock woven right in. Size & Fit - Classic Fit: Sits at the natural waist and trim, with a traditional straight leg. Fabric & Care - 100% nylon wicks moisture and resists wrinkles. Built-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. Additional Features - Button front with zip fly. Adjustable nylon belt. Two front slash pockets, two cargo pockets, two rear pockets. Zippered security pocket. Elastic inserts in waist. Imported.Sun Protection That Won't Wear Off - Our high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays.", metadata={'source': 'OutdoorClothingCatalog_3.csv', 'row': 3})

In [19]:
retriever = db.as_retriever()

In [20]:
llm = AzureChatOpenAI(temperature=0.0, openai_api_version="2023-12-01-preview", azure_deployment="gpt-35-turbo-felix", openai_api_key=os.getenv("AZURE_OPENAI_KEY"))



/Users/felixjin/.pyenv/versions/3.12.2/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [21]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

print(qdocs)

: 319
name: Men's Tropical Breeze Pants
description: These Men's cargo hiking pants are adventure-ready in a cool, moisture-wicking fabric that has 50+ UPF sunblock woven right in. Size & Fit - Classic Fit: Sits at the natural waist and trim, with a traditional straight leg. Fabric & Care - 100% nylon wicks moisture and resists wrinkles. Built-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. Additional Features - Button front with zip fly. Adjustable nylon belt. Two front slash pockets, two cargo pockets, two rear pockets. Zippered security pocket. Elastic inserts in waist. Imported.Sun Protection That Won't Wear Off - Our high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays.: 2
name: Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece
description: She'll love the bright colors, ruffles and exclusive whimsical prints of this toddler's two-piece swimsuit! Our four-way-stretch and chlorine-resistant fabric keeps it

In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
pants with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your pants with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))